In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import pandas as pd
import fiddler as fdl

In [2]:
cassandra.__version__

'3.28.0'

In [4]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
with open("datastax_auth/danny@fiddler.ai-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [33]:
rows = session.execute('SELECT * FROM fiddler_doc_snippets_openai')
    
df_docs = rows._current_rows
df_docs

,row_id,attributes_blob,body_blob,metadata_s,vector
0,992f28c79fa541a6a0a58b4eafc86921,None,"---\ntitle: ""client.delete_webhook""\nslug: ""cl...",None,"[-0.0026160315610468388, -0.026444554328918457..."
1,d03e2f3fe2514a4c8a52229fb6b13f48,None,"slug: ""publishing-events-with-complex-data-for...",None,"[-0.02673957124352455, 0.006959426216781139, -..."
2,3ea3314d6ebd4395ab423dab14f9b6a2,None,"---\ntitle: ""Class-Imbalanced Data""\nslug: ""cl...",None,"[-0.015294374898076057, 0.006210180465131998, ..."
3,08798909cdb94a6992590d78eda3ab0f,None,"slug: ""data-drift"" e-Monitor_DriftAnaly.png ""M...",None,"[-0.031133130192756653, -0.007197968661785126,..."
4,8fd0b121342142a397bd602bd8a78da9,None,"---\ntitle: ""Useful Queries for Root Cause Ana...",None,"[0.008876001462340355, 0.027711128816008568, -..."
...,...,...,...,...,...
310,e2ff7d1601934e59aa6e2383b936bbaa,None,"---\ntitle: ""Data Drift""\nslug: ""data-drift""\n...",None,"[-0.034394051879644394, -0.009222312830388546,..."
311,9e11d5e6645640608effb1001ea230b0,None,"slug: ""point-explanations"" prediction differe...",None,"[-0.020307065919041634, 0.021863754838705063, ..."
312,b50f8505df554a32bd58a530d057b61b,None,"---\ntitle: ""fdl.RowDataSource""\nslug: ""fdlrow...",None,"[-0.02189440280199051, 0.025069715455174446, -..."
313,f36dee885b324c7490ce16b13a8bc90d,None,"slug: ""class-imbalance-monitoring-example"" ler...",None,"[-0.0004906170070171356, -0.0091997180134058, ..."


In [34]:
rows = session.execute('SELECT row_id, session_id, source_docs, response, question, comment, feedback, ts FROM fiddler_chatbot_history')
    
df = rows._current_rows
df

,row_id,session_id,source_docs,response,question,comment,feedback,ts
0,c2d6c0ae-8870-4f1b-9d53-6ae1f10cd5e5,dfb9af39-ed61-4a3f-9f69-3c6a494e7675,"Document: slug: ""performance-tracking-platf...",You can track various performance metrics for ...,what kind of performance tracking can I do?,None,NaN,2023-10-13 15:52:04.240
1,feee7fdb-db14-46b8-a149-3502a1329df1,None,"Document: ---\ntitle: ""About the Fiddler Cl...",Fiddler is a company that provides an AI Obser...,Who is Fiddler?,None,NaN,2023-10-12 13:46:36.185
2,ac52a4da-bd72-40e5-850c-5d2b7e771590,4e17bdc9-dec8-4b92-adaa-388abef6eded,"Document: slug: ""ml-flow-integration"" expl...",I could not find an answer. Join our [Slack co...,Why couldn''t you find the answer?,None,NaN,2023-10-13 16:53:05.228
3,c710094f-c235-4967-a576-6e94b8f61c70,b36feb54-7464-482c-ad6c-71edd91d1585,Document: Re-uploading in Fiddler essential...,"To upload multiple events, you can use the `cl...",How can I upload multiple events?,None,NaN,2023-10-12 14:51:34.137
4,af539262-c796-4f08-b985-b0ab85039cac,7797c58c-2e0d-47ce-be28-c3680b6b706e,"Document: ---\ntitle: ""Single Sign On with ...",Okta is a single sign-on (SSO) platform that a...,what''s Okta?,None,NaN,2023-10-13 15:59:53.789
5,98950cca-94cf-4fee-9ab7-0c16c8ada4c7,b36feb54-7464-482c-ad6c-71edd91d1585,"Document: ---\ntitle: ""About Datasets""\nslu...","Currently, there isn''t a way for users to dir...",Can I delete datasets associated with a model?,None,NaN,2023-10-12 14:50:32.511
6,c5c14f05-df8c-402c-9fd3-3af8f6999231,fefc3bb2-724a-43ae-953f-651351a5a5f2,"Document: ---\ntitle: ""Baselines""\nslug: ""f...",There is no specific mention of the maximum nu...,How many of them can I have?,None,1.0,2023-10-12 13:44:06.765
7,59d12677-3640-499c-99ca-9c6f6d332f2b,ca9f6647-2418-4f72-9abc-851ba17a0383,"Document: slug: ""ml-flow-integration"" expl...",I could not find an answer.\nJoin our [Slack c...,Does this work?,None,NaN,2023-10-13 15:56:05.873
8,c30f1797-a71a-40f0-b27b-9cd6b9c22783,4e17bdc9-dec8-4b92-adaa-388abef6eded,"Document: slug: ""ml-flow-integration"" expl...",I could not find an answer.\nJoin our [Slack c...,Is this faster?,None,NaN,2023-10-13 16:52:47.922
9,8420f35f-14d8-4783-a3c6-1caffc62209d,d5a4f9af-63b2-4199-b392-17dcfdff4f93,"Document: ---\ntitle: ""System Architecture""...",I could not find an answer.\nJoin our [Slack c...,What’s on Fiddler roadmap?,None,NaN,2023-10-12 14:53:27.549


In [16]:
#session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

In [35]:
#session.execute("DROP TABLE fiddler_chatbot_history")

In [23]:
#session.execute("TRUNCATE TABLE fiddler_doc_snippets_openai")

In [27]:
session.execute("TRUNCATE TABLE fiddler_chatbot_history")